In [2]:
import torch
from torch import nn
import torch.nn.functional as F

from torchvision import datasets, transforms

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((0.5,), (0.5,)),
                               ])

# Download and load MNIST data
trainset = datasets.MNIST("~/.pytorch/MNIST_data/", download=True, train=True, transform = transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

0it [00:00, ?it/s]

9920512it [00:06, 1652040.03it/s]                            


Extracting /root/.pytorch/MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 143177.06it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /root/.pytorch/MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 2420898.35it/s]                           
0it [00:00, ?it/s]

Extracting /root/.pytorch/MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 54099.00it/s]            

Extracting /root/.pytorch/MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [0]:
# Sequential model
model = nn.Sequential(nn.Linear(784, 128),
                      nn.ReLU(),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Linear(64, 10))


#loss
criterion = nn.CrossEntropyLoss()

In [11]:
# Get data
images, labels = next(iter(trainloader))
print(images.shape)
# Reshape
images = images.view(images.shape[0], -1)
print(images.shape)

torch.Size([64, 1, 28, 28])
torch.Size([64, 784])


In [0]:
# Forward-pass
logits = model(images)

# calculate loss
loss = criterion(logits, labels)

In [13]:
loss

tensor(2.3092, grad_fn=<NllLossBackward>)

# AutoGrad


In [3]:
x = torch.randn(2,2, requires_grad=True)
print(x)

tensor([[ 0.3859, -2.3576],
        [ 0.1533,  0.1481]], requires_grad=True)


In [4]:
y = x**2
print(y)

tensor([[0.1489, 5.5583],
        [0.0235, 0.0219]], grad_fn=<PowBackward0>)


In [5]:
print(y.grad_fn)

In [6]:
z = y.mean()
print(z)

tensor(1.4382, grad_fn=<MeanBackward0>)


In [8]:
y

tensor([[0.1489, 5.5583],
        [0.0235, 0.0219]], grad_fn=<PowBackward0>)

In [9]:
z.backward()
print(x.grad)
print(x/2)

tensor([[ 0.1929, -1.1788],
        [ 0.0766,  0.0740]])
tensor([[ 0.1929, -1.1788],
        [ 0.0766,  0.0740]], grad_fn=<DivBackward0>)


# Loss and Autograd

In [15]:
print('Before backward pass: \n', model[0].weight.grad)

loss.backward()

print('After backward pass: \n', model[0].weight.grad)

Before backward pass: 
 tensor([[-0.0061, -0.0061, -0.0061,  ..., -0.0061, -0.0061, -0.0061],
        [-0.0033, -0.0033, -0.0033,  ..., -0.0033, -0.0033, -0.0033],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0023,  0.0023,  0.0023,  ...,  0.0023,  0.0023,  0.0023],
        [ 0.0007,  0.0007,  0.0007,  ...,  0.0007,  0.0007,  0.0007],
        [-0.0026, -0.0026, -0.0026,  ..., -0.0026, -0.0026, -0.0026]])


RuntimeError: ignored

# Training

In [0]:
from torch import optim

optimizer = optim.SGD(model.parameters(), lr = 0.01)